In [1]:
import  os

In [2]:
%pwd

'c:\\Users\\ASUS\\Desktop\\loan-pay-back\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ASUS\\Desktop\\loan-pay-back'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_dir: Path
    test_data_dir: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from src.loan_payment_prediction.constants import*
from src.loan_payment_prediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_dir=config.train_data_dir,
            test_data_dir=config.test_data_dir,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import numpy as np
import os
from src.loan_payment_prediction import logger
from sklearn.linear_model import ElasticNet
import joblib


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        # Load training data
        train_data = pd.read_csv(self.config.train_data_dir)
        test_data = pd.read_csv(self.config.test_data_dir)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Initialize and train the model
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Save the trained model
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"Model saved at {model_path}")

        return model

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()
except Exception as e:
    raise e

[2025-12-18 12:23:11,951]: INFO: YAML file config\config.yaml loaded successfully.
[2025-12-18 12:23:11,958]: INFO: YAML file params.yaml loaded successfully.
[2025-12-18 12:23:11,961]: INFO: YAML file schema.yaml loaded successfully.
[2025-12-18 12:23:11,964]: INFO: Directory created at: artifacts


ValueError: could not convert string to float: 'Male'